In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr
from src.DataLoader import TableDataset
from functools import partial

from src.tools import *
from src.trainer import Trainer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
BATCH_SIZE = 1000

In [3]:
dataset = TableDataset('../data/data_fe.csv')

In [4]:
# split train data to train and validation parts
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [5]:
train_dataset, val_dataset = balance_val_split(dataset)

In [6]:
train_dl = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True
)

In [7]:
model = nn.Sequential(
    nn.Linear(120, 240),
    nn.ReLU(),
    nn.Linear(240, 500),
    nn.ReLU(),
    nn.Linear(500, 240),
    nn.ReLU(),
    nn.Linear(240, 120),
    nn.ReLU(),
    nn.Linear(120, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)

In [8]:
loss = Loss_class(nn.CrossEntropyLoss())
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=model,
                    device=device)
optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.9)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=10)

wandb_init_params = dict(
    name='test_dnn_1',
    project="Internship_project"
)

In [9]:
trainer = Trainer(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params
                  )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


In [10]:
trainer.train_model(learning_params)
wandb.finish()

Let's see how good I am...: 100%|██████████| 7/7 [00:03<00:00,  2.19it/s]


Epoch: 1 of 10, 0.084 min


Let's see how good I am...: 100%|██████████| 7/7 [00:03<00:00,  2.00it/s]


Epoch: 2 of 10, 0.087 min


Let's see how good I am...: 100%|██████████| 7/7 [00:03<00:00,  2.15it/s]


...

Let's see how good I am...: 100%|██████████| 7/7 [00:03<00:00,  2.31it/s]


Epoch: 9 of 10, 0.079 min


Let's see how good I am...: 100%|██████████| 7/7 [00:03<00:00,  2.13it/s]


Epoch: 10 of 10, 0.084 min


epoch,▁▂▃▃▄▅▆▆▇█
lr,█▇▆▅▄▃▃▂▁▁
test_accuracy,▁▅▁███████
test_loss,█▁▁▁▁▁▁▁▁▁
train_accuracy,▃▆▂▁██████
train_loss,█▁▁▁▁▁▁▁▁▁
epoch,9
lr,0.00349
test_accuracy,0.96614
test_loss,0.62063
train_accuracy,0.966
